# Uses interaction to push triples to the brain

Before running, start GraphDB and make sure that there is a sandbox repository

In [1]:
# general imports for EMISSOR and the BRAIN
import emissor as em
from cltl import brain
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario
from cltl.brain.long_term_memory import LongTermMemory
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.reply_generation.lenka_replier import LenkaReplier

import os
import time
import uuid
from datetime import datetime
from datetime import date
import requests


/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import sys
import os
src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import util.driver_util as d_util
import util.capsule_util as c_util
import intentions.talk as talk
import dummies.text_to_triple as ttt

In [3]:
from random import getrandbits

##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "../../data"

### Define the folder where the images are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"


### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

Directory  ../../data/2021-11-01-21:48:54  Created 
Directory  ../../data/2021-11-01-21:48:54/image  Created 


In [4]:
import pathlib
log_path=pathlib.Path('./logs')
print(type(log_path))
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                           log_dir=log_path,
                           clear_all=True)
replier = LenkaReplier()

<class 'pathlib.PosixPath'>
2021-11-01 21:48:57,226 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-01 21:48:58,948 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents
2021-11-01 21:48:58,948 -    DEBUG -   cltl.reply_generation.api.LenkaReplier - Booted


In [6]:
#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = "Hi there. Who are you " + HUMAN_NAME + "?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

utterance = input('\n')
print(HUMAN_NAME + ": " + utterance)
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)
    
    HUMAN_NAME = ttt.get_your_name(utterance)
    HUMAN_ID = HUMAN_NAME
    
    thoughts = talk.process_text_and_reply_hacked (scenario, place_id, location, HUMAN_ID, textSignal, replier, my_brain)
    
    print(AGENT + ": " + thoughts)
    textSignal = d_util.create_text_signal(scenario, thoughts)
    scenario.append_signal(textSignal)

    ###### Getting the next input signals
    utterance = input('\n')


Leolani2: Hi there. Who are you Who_Knows_Selene??



 I am Piek


Who_Knows_Selene?: I am Piek
2021-11-01 21:51:39,173 -     INFO - cltl.triple_extraction.api.Chat (I_Am_Piek)         000 - << Start of Chat with I_Am_Piek >>
2021-11-01 21:51:40,096 -     INFO - cltl.triple_extraction.api.Chat (I_Am_Piek)         001 -  I_Am_Piek: "I am Piek"
2021-11-01 21:51:40,137 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: i-am-piek_stative_piek [agent_->_person])
2021-11-01 21:51:40,143 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Statement Novelty: 1 times, e.g. i-am-piek on November,2021
2021-11-01 21:51:41,778 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. be-ancestor-of person - 15 gaps as object: e.g. cook-by food
Leolani2: That rings a bell. i am piek told me about it in 2021 11 01T00:00:00



 Who knows Selene?


2021-11-01 21:51:55,386 -     INFO - cltl.triple_extraction.api.Chat (Who_Knows_Selene?) 000 - << Start of Chat with Who_Knows_Selene? >>
2021-11-01 21:51:56,257 -     INFO - cltl.triple_extraction.api.Chat (Who_Knows_Selene?) 001 - Who_Knows_Selene?: "Who knows Selene"


TypeError: can only concatenate str (not "list") to str

In [6]:
#scenario.scenario.end = datetime.now().microsecond
scenarioStorage.save_scenario(scenario)

## End of notebook